In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from time import time 
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import seaborn as sns
%matplotlib inline

In [2]:
# Load training data as dataframe
df = pd.read_csv('final_train.csv')

display(df)

,Unnamed: 0,Activity,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),...,tGravityAccMag-energy(),tGravityAccMag-entropy(),tGravityAccMag-iqr(),tGravityAccMag-mad(),tGravityAccMag-max(),tGravityAccMag-mean(),tGravityAccMag-min(),tGravityAccMag-sma(),tGravityAccMag-std(),void()
0,0,STANDING,-1.574191,-0.805700,-2.453831,-1.861714,0.115517,-2.023369,3.125860,-9.474436,...,5.049130,3.272281,5.103018,-0.823302,3.063681,-0.928676,-1.007695,3.094236,1.669987,1.0
1,1,WALKING_UPSTAIRS,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,...,4.565443,4.440779,3.288399,-2.689605,1.705341,-0.822617,-0.572410,-2.187890,-0.544652,2.0
2,2,LAYING,-1.548333,-0.139628,-2.380889,-1.915239,0.067613,-0.887342,6.007361,-11.906904,...,5.059609,2.296603,5.344174,-0.549843,3.272225,-0.940610,-1.031522,-2.071755,2.024442,3.0
3,3,WALKING,-1.574094,-0.793443,-2.449959,-1.842741,-0.195625,-0.950671,2.104353,-11.906121,...,4.860758,4.177535,3.950012,-0.322642,2.366287,-0.866937,-0.696431,1.863375,0.550414,2.0
4,4,LAYING,-1.548333,-0.139628,-2.380889,NaN,0.067613,-0.887342,6.007361,-11.970485,...,5.059609,2.296603,5.344174,-0.549843,3.272225,-0.940610,-1.031522,-2.102513,2.024442,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11081,11081,WALKING_UPSTAIRS,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,...,4.565443,4.440779,3.288399,-2.689605,1.705341,-0.822617,-0.572410,3.534816,-0.544652,2.0
11082,11082,LAYING,-1.550288,-0.231565,-2.376561,-1.868524,0.007653,-0.077020,5.494466,-5.860692,...,5.048737,2.923856,5.182026,-0.998495,2.853755,-0.933561,-1.000173,-1.984575,1.372763,1.0
11083,11083,WALKING_UPSTAIRS,-1.569815,-0.794940,-2.482325,-1.784151,0.021844,-0.740329,8.005261,-11.916107,...,4.453514,4.302852,2.979961,-6.032382,1.853029,-0.802817,0.041975,3.427548,-0.075184,1.0
11084,11084,WALKING_DOWNSTAIRS,-1.574768,-0.752847,-2.449529,-1.737755,-0.189761,-0.454246,8.054443,-5.611277,...,4.615939,4.443277,3.208597,-6.200222,1.675269,-0.828934,-0.877004,3.385405,-0.383217,3.0


In [3]:
# Correct the order of values and label
del df["Unnamed: 0"]

# Calculate correlation matrix between columns
cor_matrix = df.corr().abs()

# Drop highest correlated columns
#upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
#to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]
#df_analysis = df.drop(df[to_drop], axis=1) 
df_analysis = df

In [4]:
display(df_analysis)

,Activity,"angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyAccMean,gravity)","angle(tBodyGyroJerkMean,gravityMean)","angle(tBodyGyroMean,gravityMean)",energy-mean(),"fBodyAcc-bandsEnergy()-1,16",...,tGravityAccMag-energy(),tGravityAccMag-entropy(),tGravityAccMag-iqr(),tGravityAccMag-mad(),tGravityAccMag-max(),tGravityAccMag-mean(),tGravityAccMag-min(),tGravityAccMag-sma(),tGravityAccMag-std(),void()
0,STANDING,-1.574191,-0.805700,-2.453831,-1.861714,0.115517,-2.023369,3.125860,-9.474436,3.622600,...,5.049130,3.272281,5.103018,-0.823302,3.063681,-0.928676,-1.007695,3.094236,1.669987,1.0
1,WALKING_UPSTAIRS,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,2.512880,...,4.565443,4.440779,3.288399,-2.689605,1.705341,-0.822617,-0.572410,-2.187890,-0.544652,2.0
2,LAYING,-1.548333,-0.139628,-2.380889,-1.915239,0.067613,-0.887342,6.007361,-11.906904,3.621984,...,5.059609,2.296603,5.344174,-0.549843,3.272225,-0.940610,-1.031522,-2.071755,2.024442,3.0
3,WALKING,-1.574094,-0.793443,-2.449959,-1.842741,-0.195625,-0.950671,2.104353,-11.906121,3.355701,...,4.860758,4.177535,3.950012,-0.322642,2.366287,-0.866937,-0.696431,1.863375,0.550414,2.0
4,LAYING,-1.548333,-0.139628,-2.380889,NaN,0.067613,-0.887342,6.007361,-11.970485,3.621984,...,5.059609,2.296603,5.344174,-0.549843,3.272225,-0.940610,-1.031522,-2.102513,2.024442,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11081,WALKING_UPSTAIRS,-1.568669,-0.612620,-2.488338,-1.751117,0.218134,0.280294,7.953403,-0.679894,2.512880,...,4.565443,4.440779,3.288399,-2.689605,1.705341,-0.822617,-0.572410,3.534816,-0.544652,2.0
11082,LAYING,-1.550288,-0.231565,-2.376561,-1.868524,0.007653,-0.077020,5.494466,-5.860692,3.603169,...,5.048737,2.923856,5.182026,-0.998495,2.853755,-0.933561,-1.000173,-1.984575,1.372763,1.0
11083,WALKING_UPSTAIRS,-1.569815,-0.794940,-2.482325,-1.784151,0.021844,-0.740329,8.005261,-11.916107,2.644861,...,4.453514,4.302852,2.979961,-6.032382,1.853029,-0.802817,0.041975,3.427548,-0.075184,1.0
11084,WALKING_DOWNSTAIRS,-1.574768,-0.752847,-2.449529,-1.737755,-0.189761,-0.454246,8.054443,-5.611277,2.473134,...,4.615939,4.443277,3.208597,-6.200222,1.675269,-0.828934,-0.877004,3.385405,-0.383217,3.0


In [5]:
# Divide dataframe into values and labels
Y_labels = df_analysis['Activity'] 
X_values = df_analysis[['fBodyAcc-meanFreq()-Z', 'fBodyAccJerk-energy()-Y', 'fBodyAccJerk-max()-Y', 
                        'fBodyAccMag-mad()', 'tGravityAcc-min()-X', 'fBodyAccJerk-std()-Y', 
                        'tBodyGyroJerk-mad()-X', 'fBodyGyro-bandsEnergy()-41,48', 'tBodyAcc-iqr()-X', 
                        'fBodyAccJerk-bandsEnergy()-33,40.2', 'tGravityAcc-mean()-X', 'fBodyAccMag-std()', 
                        'fBodyAccJerk-bandsEnergy()-1,24.2', 'fBodyAccJerk-mad()-Y', 'angle(X,gravityMean)', 
                        'tBodyAccJerk-std()-Y', 'angle(Y,gravityMean)', 'tBodyAcc-max()-X', 
                        'fBodyAccJerk-meanFreq()-Z', 'fBodyAcc-max()-X', 'fBodyAcc-maxInds-X', 
                        'tBodyAcc-std()-X', 'tGravityAcc-max()-Y', 'tBodyAccMag-mad()', 
                        'tGravityAcc-energy()-X', 'tBodyGyro-max()-X', 'tBodyAcc-mad()-X', 
                        'tBodyAcc-energy()-X', 'fBodyAcc-max()-Z', 'tBodyAcc-energy()-Y', 
                        'tBodyAccMag-max()', 'fBodyGyro-max()-Z', 'fBodyAccJerk-bandsEnergy()-17,32.2', 
                        'tGravityAcc-arCoeff()-X,2']]
# Wersja z 157 labelami
#X_values = df_analysis.loc[:, df_analysis.columns != 'Activity']

# Save columns names 
variables = list(X_values.columns.values)
display(X_values)

,fBodyAcc-meanFreq()-Z,fBodyAccJerk-energy()-Y,fBodyAccJerk-max()-Y,fBodyAccMag-mad(),tGravityAcc-min()-X,fBodyAccJerk-std()-Y,tBodyGyroJerk-mad()-X,"fBodyGyro-bandsEnergy()-41,48",tBodyAcc-iqr()-X,"fBodyAccJerk-bandsEnergy()-33,40.2",...,tGravityAcc-energy()-X,tBodyGyro-max()-X,tBodyAcc-mad()-X,tBodyAcc-energy()-X,fBodyAcc-max()-Z,tBodyAcc-energy()-Y,tBodyAccMag-max(),fBodyGyro-max()-Z,"fBodyAccJerk-bandsEnergy()-17,32.2","tGravityAcc-arCoeff()-X,2"
0,-1.953725,2.334721,1.764781,2.454973,-2.151051,0.585073,6.869111,1.213230,3.007449,2.185818,...,1.531084,1.141714,1.688795,5.575829,0.495253,1.877528,5.427825,2.500052,-2.220692,4.210923
1,-1.886313,2.308206,1.905332,0.851388,-2.187197,1.293797,6.290559,1.210855,2.693488,2.167196,...,1.585023,-0.220436,0.891455,5.146982,0.455411,1.814248,3.509034,1.729258,-2.222886,3.661973
2,-2.008311,2.338379,1.714682,2.642219,-2.500477,0.303414,6.952255,1.213895,3.003237,2.192577,...,1.851578,1.783924,1.689827,5.575877,0.510324,1.882485,5.722415,2.566368,-2.220568,3.152277
3,-1.925148,2.283781,1.947702,1.618935,-2.153766,1.763192,6.217321,1.212512,2.860743,2.184860,...,1.532973,0.289834,1.333694,5.472212,0.462394,1.806945,4.442686,2.383545,-2.221784,3.389088
4,-2.008311,2.338379,1.714682,2.642219,-2.500477,0.303414,6.952255,1.213895,3.003237,2.192577,...,1.851578,1.783924,1.689827,5.575877,0.510324,1.882485,5.722415,2.566368,-2.220568,3.152277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11081,-1.886313,2.308206,1.905332,0.851388,-2.187197,1.293797,6.290559,1.210855,2.693488,2.167196,...,1.585023,-0.220436,0.891455,5.146982,0.455411,1.814248,3.509034,1.729258,-2.222886,3.661973
11082,-1.929330,2.337553,1.727461,2.204386,-2.461722,0.390518,6.881634,1.213792,3.002903,2.192855,...,1.839423,1.180480,1.644670,5.568437,0.506342,1.878288,5.131284,2.418802,-2.220600,2.992427
11083,-1.896370,2.268223,1.995122,1.265390,-2.190643,1.706158,6.365741,1.209718,2.700524,2.140280,...,1.569838,-0.938951,0.962617,5.217194,0.409625,1.748575,3.717658,1.485592,-2.222579,2.990591
11084,-1.901412,2.262047,2.109786,0.816508,-2.162445,1.996001,6.016609,1.212969,2.614537,2.183555,...,1.538489,-1.685190,0.886699,5.162105,0.461533,1.759468,3.466554,2.163630,-2.221080,2.949921


In [6]:
# Standardising the data
# Center to the mean and component wise scale to unit variance
# for col in [variables]:
#     X_all[col] = scale(X_all[col])
sc = StandardScaler()
X_values = sc.fit_transform(X_values)

In [7]:
# Dividing data into training and test set taking 80% of prior data as a training set,
# so that our models are tested based on matches that took place after matches from our training set
X_train, X_test= np.split(X_values, [int(.8 *len(X_values))])
Y_train, Y_test= np.split(Y_labels, [int(.8 *len(Y_labels))])

In [8]:
# Printing shape of our data sets
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', Y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)

Training Features Shape: (8868, 34)
Training Labels Shape: (8868,)
Testing Features Shape: (2218, 34)
Testing Labels Shape: (2218,)


In [9]:
# Training the classifier and measuring its time
def train(clf, X_train, Y_train):
    
    # Starting the clock, training the classifier, then stoping the clock
    start = time()
    clf.fit(X_train, Y_train)
    end = time()
    
    # Printing the results
    print ("Model trained in: ",end - start)

In [10]:
# Making predictions 
def predict(clf, features, target):
    
    # Starting the clock, making predictions, then stoping the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Printing results
    print ("Prediction made in: ",end - start)
    
    # Returning F1 score and accuracy
    return f1_score(target, y_pred, average=None), sum(target == y_pred) / float(len(y_pred)), y_pred

In [11]:
# Evaluation of our model
def fit(clf, X_train, y_train, X_test, y_test):
    
    # Indicating the model and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Training model
    train(clf, X_train, y_train)
    
    # Printing the results of prediction for both training and testing set
    f1, acc, predictions = predict(clf, X_train, y_train)
    print ("===========================================")
    print ("Indexes assesing methods for training set:")
    print ("F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]:", f1)
    print ("Accuracy:", acc)
    
    print ("===========================================")
    f1, acc, predictions = predict(clf, X_test, y_test)
    print ("Indexes assesing methods for test set:")
    print ("F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]:", f1)
    print ("Accuracy:", acc)

In [12]:
# Inference function
def inference(clf_XGB, inf):
    start = time()
    y_pred = clf_XGB.predict(inf)
    end = time()

    # Printing results
    print ("Inference made in: ",end - start)

In [13]:
# Creating our model and training it
clf_XGB = xgb.XGBClassifier()

fit(clf_XGB, X_train, Y_train, X_test, Y_test)
print ('')

Training a XGBClassifier using a training set size of 8868. . .


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:30:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Model trained in:  8.20032787322998
Prediction made in:  0.0370023250579834
Indexes assesing methods for training set:
F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]: [1. 1. 1. 1. 1. 1.]
Accuracy: 1.0
Prediction made in:  0.007999897003173828
Indexes assesing methods for test set:
F1 score [LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS]: [1.         0.96666667 0.96199214 0.97894737 0.98305085 0.99152542]
Accuracy: 0.9801623083859333



In [14]:
# Tunning
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# Defining parameters we want to tune
parameters = { 'learning_rate' : [0.3], 
              'n_estimators' : [150], 
              'max_depth': [5], 
              'min_child_weight': [3], 
              'gamma':[0.6], 
              'subsample' : [0.8], 
              'colsample_bytree' : [0.7] }

# Initializing the classifier
clf_XGB = xgb.XGBClassifier(seed=2)

# Performing grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf_XGB,
                        scoring='f1_macro',
                        param_grid=parameters,
                        cv=5,
                        verbose=1)

grid_obj = grid_obj.fit(X_train,Y_train)

clf_XGB = grid_obj.best_estimator_

# Printing out best found parameters
grid_obj.best_params_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[05:30:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:30:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:30:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:30:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:30:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   29.0s finished


[05:30:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


{'colsample_bytree': 0.7,
 'gamma': 0.6,
 'learning_rate': 0.3,
 'max_depth': 5,
 'min_child_weight': 3,
 'n_estimators': 150,
 'subsample': 0.8}

In [15]:
# Inference time
pred_try = np.array([X_train[0]])
inference(clf_XGB, pred_try)

Inference made in:  0.002000570297241211


In [16]:
# Saving trained model
clf_XGB.save_model('XGB_with_tunning.model')

In [17]:
# Loading trained model
clf2 = xgb.XGBClassifier()
clf2.load_model('XGB_with_tunning.model')

In [18]:
f1f, accf, predicted = predict(clf2, X_values, Y_labels)
print("Accuracy after tuning: ", accf)

Prediction made in:  0.04599809646606445
Accuracy after tuning:  0.9956702146851886


In [19]:
np.mean(predicted == df.Activity)

0.9956702146851886